In [215]:
# Connect to Flask API

import requests
from IPython.display import display, HTML

AUTH = ("admin", "admin") # Development authentication
ENDPOINT = "https://api:5000" # Accessible inside Docker network

response = requests.get(ENDPOINT + "/", verify = False, auth = AUTH)

# Check if the request was successful
if response.status_code == 200:
    # Get the HTML content from the response
    html_content = response.text
    # Display the HTML content
    display(HTML(html_content))
else:
    print("Failed to retrieve HTML content:", response.status_code)

/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [219]:
response = requests.get(ENDPOINT + "/get_tables", verify = False, auth = AUTH).json()

# Check if the request was successful
if response['success'] == True:
    print(response['message'])
    print(response['data'])
else:
    print(response['error'])

Successfully retrieved all tables from 'crawldb'.
['page', 'image', 'link', 'site', 'page_type', 'page_data', 'data_type']


/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [221]:
data = {
    "domain": "pajki.com",
    "robots_content": "robots.txt",
    "sitemap_content": "sitemap.xml"
}


response = requests.post(ENDPOINT + "/insert_site", json=data, verify = False, auth = AUTH).json()

# Check if the request was successful
print(response)
# if response['success'] == True:
#     print(response['message'])
#     print(response['data'])
# else:
#     print(response['error'])

['data', 2]


/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [214]:
# Connect to Flask API

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# Check the selenium version insalled
import selenium
print(selenium.__version__)

WEB_DRIVER_LOCATION = "/app/geckodriver"
TIMEOUT = 5

import requests
from requests.adapters import HTTPAdapter

session = requests.Session()
session.mount('http://', HTTPAdapter(max_retries=1))
session.mount('https://', HTTPAdapter(max_retries=2))


#AUTH = ("admin", "admin") # Development authentication
#ENDPOINT = "https://api:5000" # Accessible inside Docker network

#response = requests.get(ENDPOINT + "/", verify = False, auth = AUTH)

# Check if the request was successful
#if response.status_code == 200:
    # Get the HTML content from the response
#    html_content = response.text
    # Display the HTML content
#else:
#    print("Failed to retrieve HTML content:", response.status_code)

SyntaxError: invalid syntax (2927940326.py, line 29)

In [37]:
class Site:
    def __init__(self, domain, robots_content, sitemap_content, crawl_delay):
        self.domain = domain
        self.robots_content = robots_content
        self.sitemap_content = sitemap_content
        self.crawl_dielay = crawl_delay
        
class Page:
    def __init__(self, url, domain, page_type_code=None, content_hash=None, http_status_code=None, accessed_time=None, data_type_code=None, html_content=None):
        self.page_type_code = page_type_code
        self.url = url
        self.html_content = html_content
        self.content_hash = content_hash
        self.http_status_code = http_status_code
        self.accessed_time = accessed_time
        self.domain = domain
        self.data_type_code = data_type_code
        
class Image:
    def __init__(self, filename, content_type, data, accessed_time):
        self.filename = filename
        self.content_type = content_type
        self.data = data
        self.accessed_time = accessed_time

In [207]:
import urllib
from urllib.parse import urlparse, urlunparse, urldefrag
from bs4 import BeautifulSoup
import threading
from queue import Queue
import urllib.robotparser
import datetime
import idna

class Crawler:
    def __init__(self, initial_seed, num_workers):
        self.initial_seed = initial_seed
        self.frontier = Queue()
        # self.visited = set()
        self.num_workers = num_workers
        self.web_driver = self.initialize_web_driver()
        # self.lock = threading.Lock()
    
    def initialize_web_driver(self):
        options = FirefoxOptions()
        options.add_argument("--headless")
        options.add_argument("user-agent=fri-wier-oskapha")
        service = Service(executable_path=WEB_DRIVER_LOCATION)
        driver = webdriver.Firefox(service=service, options=options)
        driver.set_page_load_timeout(10)
        return driver
        
    def canonicalize_url(self, url):
        """
        Canonicalizes a URL by removing redundant parts and ensuring consistency.
        """
        parsed_url = urlparse(url)
        parsed_url = parsed_url._replace(fragment='')
        parsed_url = parsed_url._replace(scheme=parsed_url.scheme.lower(), netloc=parsed_url.netloc.lower())
        parsed_url = parsed_url._replace(query='')
        
        if parsed_url.path.endswith('/'):
            parsed_url = parsed_url._replace(path=parsed_url.path[:-1])
        if parsed_url.path.endswith('/index.html'):
            parsed_url = parsed_url._replace(path=parsed_url.path[:-11])
        if parsed_url.path.endswith('/index.php'):
            parsed_url = parsed_url._replace(path=parsed_url.path[:-10])
        
        if parsed_url.scheme == 'http' or parsed_url.scheme == 'https':
            pass
        else:
            return None
        
        parsed_url = parsed_url._replace(path=urlparse(parsed_url.geturl()).path)
        
        canonicalized_url = urlunparse(parsed_url)
        
        return canonicalized_url
    
    def robots_allow(self, url):
        # Checks if crawler is allowed on a specified url
        domain = self.get_domain(url)
        domain = self.remove_leading_domain_dot_symbol(domain)
        rp = urllib.robotparser.RobotFileParser()
        rp.read()
        return rp.can_fetch(GROUP_NAME, url) # GROUP_NAME == user-agent
    
    def robots_content(self, url):
        # Gets robots content, sitemaps and cralwer delay (if it exists)
        # https://docs.python.org/3/library/urllib.robotparser.html
        try:
            domain = self.get_domain(url)
            domain = self.remove_leading_domain_dot_symbol(domain)
            rp = urllib.robotparser.RobotFileParser()
            rp.set_url(f'https://{domain}/robots.txt')
            rp.read()

            request_rate = rp.request_rate('*')  # '*' for useragent
            crawl_delay = rp.crawl_delay('*')  # '*' for useragent
            site_maps = self.parse_site_maps(rp.site_maps())

            return domain, rp, crawl_delay, request_rate, site_maps
        except Exception as e:
            print(f"Error fetching robots content for URL {url}: {e}")
            return None, None, None, None, None
    
    def get_site_maps_links(self, url):
        domain, robots_content, crawl_delay, request_rate, site_maps = self.robots_content(url)
        return self.parse_site_maps(site_maps)
    
    def parse_site_maps(self, site_maps):
        links = []
        if site_maps is not None:
            for site_map_url in site_maps:
                print("Getting urls for sitemap:", site_map_url)
                try:
                    response = requests.get(site_map_url)
                    if response.status_code == 200:
                        content_type = response.headers.get('content-type', '')
                        if 'xml' in content_type:
                            # Parse XML sitemap
                            soup = BeautifulSoup(response.content, 'xml')
                            urls = soup.find_all("url")
                            for url in urls:
                                links.append(url.find("loc").text)
                        elif 'text/plain' in content_type:
                            # Parse plain text sitemap
                            urls = response.text.split('\n')
                            links.extend(urls)
                        elif 'text/html' in content_type:
                            # Parse HTML sitemap
                            soup = BeautifulSoup(response.content, 'html.parser')
                            # Extract links from HTML content
                            # Adjust this part based on the structure of the HTML sitemap
                            urls = [a['href'] for a in soup.find_all('a', href=True)]
                            links.extend(urls)
                        else:
                            print("Unsupported sitemap format:", content_type)
                    else:
                        print(f"Failed to fetch sitemap from URL: {site_map_url}")
                except Exception as e:
                    print(f"Error fetching or parsing sitemap: {e}")
                    
        print("LINKSSSSS:", links)
        return links

    def remove_leading_domain_dot_symbol(self, domain):
        if domain.startswith("."):
            return domain[1:]
        else:
            return domain

    def check_duplicates(self, page):
        # Check if the page is already in the database by checking the content hash
        content_hashes = []
        # Get function to get the content hash of all pages in the database
        
        for content_hash in content_hashes:
            if content_hash == page.content_hash:
                return True
            else:
                return False
    
    def get_domain(self, url):
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        if "www." in domain:
            domain = domain.replace("www.", ".")
        return domain
    
    def get_frontier(self):
        # Call database to get all the frontier entries from Page table
        # For each push into frontier.Queue()
        
        print('Frontier size: %s', self.frontier.qsize())
        
    def set_frontier(self):
        print('Setting frontier')

        # Search the database
        # Pages with PAGE_TYPE = FRONTIER
        
        # self.frontier.put(page)
                
        print('Frontier size: %s', self.frontier.qsize())
        
    def initialize_seed(self):
        print('Initializing seed')
        # Add URLs to frontier passed from the argument
        # In case there are no URLs passed from the argument, search for the seed URLs in the database
        if len(self.initial_seed) != 0:
            for url in self.initial_seed:
                #  Insert page into the frontier to DB
                page = Page(url, self.get_domain(url))
                
                # Insert each page into frontier
                
                self.frontier.put(page)
                
        print('Frontier size: %s', self.frontier.qsize())
        
    def validate_links(self, links):
        validated_links = []
        for link in links:
            #print('Orginial Link: ', link)
            if link is not None and len(link) > 0 and 'javascript:' not in link.lower() and 'mailto:' not in link.lower():
                domain = self.get_domain(link)
                #print('Domain: ', domain)
                if "gov.si" in domain: # Add robots check before appending
                    # Removes /? 
                    #url = urldefrag(link)[0]
                    
                    canonicalized_url = self.canonicalize_url(link)
                    #print('Canonicalized: ', canonicalized_url)
                    
                    # Check for duplicates in the array
                    if (canonicalized_url not in validated_links or canonicalized_url not in self.initial_seed):
                        #print('Valid')
                        validated_links.append(canonicalized_url)
            
        return validated_links
        
    # Get all urls from links on the page
    def get_links(self, page):
        print('Getting links')
        links = []
        
        # Check robots again
        site_maps_links = self.get_site_maps_links(page.url)
        for site_map_link in site_maps_links:
            links.append(site_maps_link)
            print(links)
        
        # Get links from href
        try:
            hrefs = self.web_driver.find_elements(by=By.XPATH, value="//a[@href]")
            for href in hrefs:
                try:
                    href_value = href.get_attribute("href")
                    links.append(href_value)
                except StaleElementReferenceException as e:
                    pass
        except NoSuchElementException:
            print('No elements')    
        
        # Get links from buttons
        # try:
        #    buttons = self.web_driver.find_element(by=By.XPATH, value="//*[@onclick]")
        #    for button in buttons:
        #        try:
        #            click_value = button.get_attribute('onclick')
        #            # print("Button: ", button)
        #        except StaleElementReferenceException as e:
        #            pass
        #except NoSuchElementException:
        #    print('No elements')
        urls = []
        soup = BeautifulSoup(page.html_content, 'html.parser')
        for script in soup(["script", "style"]):
            script.extract()
        
        for element in soup.select("[onClick], [onclick]"):
            print('Element: ', element)
        
        validated_links = self.validate_links(links)
        
        for smt in validated_links:
            print(smt)
            
        return validated_links
                
    def crawl_webpage(self, page):
        print('Started crawling', page.url)
        
        try:
            # Check if crawler is allowed based on robots.txt and save robots content
            if (self.robots_allow(page.url)):
                domain, robots_content, crawl_delay, request_rate, site_maps = self.robots_content(page.url)
                # Initialize Site and save robots content
                site = Site(
                    domain = domain,
                    robots_content = robots_content,
                    sitemap_content = site_maps,
                    crawl_delay = crawl_delay)
                
                # Wait for crawl_delay if it exists
                if crawl_delay is not None:
                    time.sleep(crawl_delay)
            else:
                print('Not allowed to crawl: ', page.url)
                return
            
            # Read the page
            
            response = requests.head(page.url, allow_redirects=True, timeout=10)
            page.http_status_code = response.status_code
            page.accessed_time = datetime.datetime.now()
            page.content_type = response.headers['Content-Type']
            
            # Is it HTML
            print("Checking content type...")
            if "text/html" in response.headers['Content-Type']:
                page.page_type_code = "HTML"
                self.web_driver.get(page.url)
                page.html_content = self.web_driver.page_source
                
                
                # Check duplicates
                
                # Check links
                print("Checking links...")
                links = self.get_links(page)
                
                # Check for images
                
                # Insert links into frontier
                
                # Insert image data into database
                
            else:
                # Else it is a binary file
                print("It's a binary file!")
                
                page.html_content = None
                page.page_type_code = "BINARY"
                if "application/pdf" == page.content_type:
                    page.data_type = "PDF"
                    
                elif "application/msword" == page.content_type:
                    page.data_type = "DOC"
                
                elif "application/vnd.openxmlformats-officedocument.wordprocessingml.document" == page.content_type:
                    page.data_type = "DOCX"
                    
                elif "application/vnd.ms-powerpoint" == page.content_type:
                    page.data_type = "PPT"
                    
                elif "application/vnd.openxmlformats-officedocument.presentationml.presentation" == page.content_type:
                    page.data_type = "PPTX"
            
                # Insert into database table Page Data since it is a binary file
                
                # Also update the page entry in table Page
                
        except Exception as e:
            # Update database page entry with PAGE_TYPE = TIMEOUT
            print('Error crawling %s: %s', page, e)
            return

In [208]:
# Start of crawler

import urllib3
import requests
from requests.adapters import HTTPAdapter
import warnings

INITIAL_SEED = [
    'https://www.gov.si',
    'https://spot.gov.si',
    'https://e-uprava.gov.si',
    'https://www.e-prostor.gov.si'
]

NUMBER_OF_WORKERS = 1

GROUP_NAME='wier2024'

In [209]:
crawler = Crawler(initial_seed=INITIAL_SEED, num_workers=NUMBER_OF_WORKERS)
crawler.initialize_seed()

Initializing seed
Getting domain for: https://www.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.gov.si', path='', params='', query='', fragment='')
.gov.si
Getting domain for: https://spot.gov.si
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='', params='', query='', fragment='')
spot.gov.si
Getting domain for: https://e-uprava.gov.si
Parsed url: ParseResult(scheme='https', netloc='e-uprava.gov.si', path='', params='', query='', fragment='')
e-uprava.gov.si
Getting domain for: https://www.e-prostor.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.e-prostor.gov.si', path='', params='', query='', fragment='')
.e-prostor.gov.si
Frontier size: %s 4


In [210]:
while crawler.frontier.qsize() > 0:
    # Dequeue element
    element = crawler.frontier.get()
    
    # Process the element
    crawler.crawl_webpage(page=element)
    
    crawler.frontier.task_done()
    
    # Get frontier from database until frontier empty
    
print('Crawler finished')
crawler.web_driver.quit()

Started crawling https://www.gov.si
Checking if robots allows crawling on:  https://www.gov.si
Getting domain for: https://www.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.gov.si', path='', params='', query='', fragment='')
.gov.si
gov.si
set rb
set url
Read robots content:  User-agent: *
Disallow: /admin
Disallow: /resources
Disallow: /pomoc
Getting domain for: https://www.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.gov.si', path='', params='', query='', fragment='')
.gov.si
Getting urls for sitemap: https://www.gov.si/sitemap.xml
LINKSSSSS: []
Checking content type...
Checking links...
Getting links
Getting domain for: https://www.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.gov.si', path='', params='', query='', fragment='')
.gov.si
Getting urls for sitemap: https://www.gov.si/sitemap.xml
LINKSSSSS: []
LINKSSSSS: []
Getting domain for: https://www.gov.si/#content
Parsed url: ParseResult(scheme='https', netloc='www.gov.si', path='/', params=

Read robots content:  
Getting domain for: https://spot.gov.si
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='', params='', query='', fragment='')
spot.gov.si
LINKSSSSS: []
Checking content type...
Checking links...
Getting links
Getting domain for: https://spot.gov.si
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='', params='', query='', fragment='')
spot.gov.si
LINKSSSSS: []
LINKSSSSS: []
Getting domain for: https://spot.gov.si/sl/#content
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='/sl/', params='', query='', fragment='content')
spot.gov.si
Getting domain for: https://spot.gov.si/sl/
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='/sl/', params='', query='', fragment='')
spot.gov.si
Getting domain for: https://spot.gov.si/sl/
Parsed url: ParseResult(scheme='https', netloc='spot.gov.si', path='/sl/', params='', query='', fragment='')
spot.gov.si
Getting domain for: https://spot.gov.si/en/
Parsed url

Failed to fetch sitemap from URL: https://e-uprava.gov.si/sitemaps/sitemap.xml
LINKSSSSS: []
Error crawling %s: %s <__main__.Page object at 0x7f841bec1c40> 'content-type'
Started crawling https://www.e-prostor.gov.si
Checking if robots allows crawling on:  https://www.e-prostor.gov.si
Getting domain for: https://www.e-prostor.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.e-prostor.gov.si', path='', params='', query='', fragment='')
.e-prostor.gov.si
e-prostor.gov.si
set rb
set url
Read robots content:  
Getting domain for: https://www.e-prostor.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.e-prostor.gov.si', path='', params='', query='', fragment='')
.e-prostor.gov.si
LINKSSSSS: []
Checking content type...
Checking links...
Getting links
Getting domain for: https://www.e-prostor.gov.si
Parsed url: ParseResult(scheme='https', netloc='www.e-prostor.gov.si', path='', params='', query='', fragment='')
.e-prostor.gov.si
LINKSSSSS: []
LINKSSSSS: []
Element:  <button